### Making Predictions, Preparing a Kaggle Submission File

Submissions should follow the kaggle given format, which is two columns (comma-separated).
The first column is the air_store_id, which is concatenated with the visit date. The second is the predicted number of visitors to the specific restaurant.

The prediction dates are: 2017-04-23 through 2017-05-31.

Each air_store_id should have info. for each date.

This first file is a very naive prediction. It is mostly being used for a first pass and for testing. 
Here, the predicted number of visitors for each restaurant is the historical mean number of visitors to that restaurant on that weekday. It uses all visitor data available and does not have any feature engineering. Stay tuned for future parts for improvements on this!

In [1]:
#setup
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
#import missingno
from sklearn import *

aReserveDF = pd.read_csv('air_reserve.csv', parse_dates = ['visit_datetime', 'reserve_datetime']) 
aVisitDF = pd.read_csv('air_visit_data.csv', parse_dates = ['visit_date']) 
aStoreDF = pd.read_csv('air_store_info.csv')

hReserveDF = pd.read_csv('hpg_reserve.csv', parse_dates = ['visit_datetime', 'reserve_datetime']) 
hStoreDF = pd.read_csv('hpg_store_info.csv') 

dateInfoDF = pd.read_csv('date_info.csv', parse_dates = ['calendar_date'])

sampleSubmissionDF = pd.read_csv('sample_submission.csv') 

storeIdRelationDF = pd.read_csv('store_id_relation.csv') 
hReserveDF['visit_year'] = hReserveDF['visit_datetime'].dt.year
hReserveDF['visit_month'] = hReserveDF['visit_datetime'].dt.month
hReserveDF['visit_day'] = hReserveDF['visit_datetime'].dt.day
hReserveDF['reserve_year'] = hReserveDF['reserve_datetime'].dt.year#
hReserveDF['reserve_month'] = hReserveDF['reserve_datetime'].dt.month
hReserveDF['reserve_day'] = hReserveDF['reserve_datetime'].dt.day
#hReserveDF.drop(['visit_datetime','reserve_datetime'], axis=1, inplace=True)

hReserveDF = hReserveDF.groupby(['hpg_store_id', 'visit_year', 'visit_month','visit_day','reserve_year','reserve_month','reserve_day', 'reserve_datetime', 'visit_datetime'], as_index=False).sum()
aReserveDF['visit_year'] = aReserveDF['visit_datetime'].dt.year
aReserveDF['visit_month'] = aReserveDF['visit_datetime'].dt.month
aReserveDF['visit_day'] = aReserveDF['visit_datetime'].dt.day
aReserveDF['reserve_year'] = aReserveDF['reserve_datetime'].dt.year
aReserveDF['reserve_month'] = aReserveDF['reserve_datetime'].dt.month
aReserveDF['reserve_day'] = aReserveDF['reserve_datetime'].dt.day

#aReserveDF.drop(['visit_datetime','reserve_datetime'], axis=1, inplace=True)
dateInfoDF['calendar_year'] = dateInfoDF['calendar_date'].dt.year
dateInfoDF['calendar_month'] = dateInfoDF['calendar_date'].dt.month
dateInfoDF['calendar_day'] = dateInfoDF['calendar_date'].dt.day
#dateInfoDF.drop(['calendar_date'], axis=1, inplace=True)
aVisitDF['visit_year'] = aVisitDF['visit_date'].dt.year
aVisitDF['visit_month'] = aVisitDF['visit_date'].dt.month
aVisitDF['visit_day'] = aVisitDF['visit_date'].dt.day
aVisitDF.drop(['visit_date'], axis=1, inplace=True)

hReserveDF = pd.merge(hReserveDF, storeIdRelationDF, on='hpg_store_id', how='inner')
hReserveDF.drop(['hpg_store_id'], axis=1, inplace=True)
aReserveDF = pd.concat([aReserveDF, hReserveDF])
aReserveDF = aReserveDF.groupby(['air_store_id', 'visit_year', 'visit_month','visit_day', 'visit_datetime', 'reserve_datetime'],\
                         as_index=False).sum().drop(['reserve_day','reserve_month','reserve_year'], axis=1)
aReserveDF = pd.merge(aReserveDF, dateInfoDF, left_on=['visit_year','visit_month','visit_day'], right_on=['calendar_year','calendar_month','calendar_day'], how='left')
aReserveDF.drop(['calendar_year','calendar_month','calendar_day'], axis=1, inplace=True)
aReserveDF = pd.merge(aReserveDF, aStoreDF, on='air_store_id', how='left')
trainDF = pd.merge(aReserveDF, aVisitDF, on=['air_store_id','visit_year','visit_month','visit_day'], how='left')
trainDF.fillna(0,inplace=True)

trainDF.sort_values(by=['visit_year','visit_month', 'visit_day', 'air_store_id'],ascending=[True,True,True,True],inplace=True)
grouped=trainDF.groupby(['visit_year','visit_month', 'visit_day','air_store_id','visitors', 'day_of_week', 'holiday_flg', 'air_genre_name','latitude','longitude'], as_index=False)['reserve_visitors'].sum()

grouped['day_of_week'] = grouped['day_of_week'].astype('category')
grouped['day_of_week_codes'] = grouped['day_of_week'].cat.codes
grouped = grouped.loc[grouped['visit_year']==2017]
grouped2 = grouped.groupby(['air_store_id', 'day_of_week_codes','air_genre_name','latitude','longitude'])['visitors'].mean()


/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:913: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/learning_curve.py:

In [2]:
grouped2 = grouped2.to_frame().reset_index()
grouped2['air_genre_name']=grouped2['air_genre_name'].astype('category')
grouped2['air_genre_name']=grouped2['air_genre_name'].cat.codes
grouped2.head()

,air_store_id,day_of_week_codes,air_genre_name,latitude,longitude,visitors
0,air_00a91d42b08b08d9,0,6,35.694003,139.753595,39.333333
1,air_00a91d42b08b08d9,1,6,35.694003,139.753595,18.000000
2,air_00a91d42b08b08d9,2,6,35.694003,139.753595,13.500000
3,air_00a91d42b08b08d9,4,6,35.694003,139.753595,31.166667
4,air_00a91d42b08b08d9,5,6,35.694003,139.753595,29.428571


In [3]:
groupedmedian = grouped.groupby(['air_store_id', 'day_of_week_codes'])['visitors'].median()
groupedmedian = groupedmedian.to_frame().reset_index() 

In [4]:
groupedmedian.rename(columns={'visitors':'median_visitors'},inplace=True)
groupedmedian.head()

,air_store_id,day_of_week_codes,median_visitors
0,air_00a91d42b08b08d9,0,38.0
1,air_00a91d42b08b08d9,1,18.0
2,air_00a91d42b08b08d9,2,14.5
3,air_00a91d42b08b08d9,4,33.0
4,air_00a91d42b08b08d9,5,28.0


In [5]:
grouped3 = grouped.groupby(['air_store_id', 'day_of_week_codes'])['reserve_visitors'].mean()
grouped3 = grouped3.to_frame().reset_index()
grouped3.head()
grouped2= grouped2.merge(grouped3,on =['air_store_id','day_of_week_codes'], how='left')

In [6]:
grouped2 = grouped2.merge(groupedmedian,on=['air_store_id','day_of_week_codes'],how='left')
grouped2.head()

,air_store_id,day_of_week_codes,air_genre_name,latitude,longitude,visitors,reserve_visitors,median_visitors
0,air_00a91d42b08b08d9,0,6,35.694003,139.753595,39.333333,3.000000,38.0
1,air_00a91d42b08b08d9,1,6,35.694003,139.753595,18.000000,19.000000,18.0
2,air_00a91d42b08b08d9,2,6,35.694003,139.753595,13.500000,7.750000,14.5
3,air_00a91d42b08b08d9,4,6,35.694003,139.753595,31.166667,5.500000,33.0
4,air_00a91d42b08b08d9,5,6,35.694003,139.753595,29.428571,4.571429,28.0


### Create submission file

The submission file should be in the format as seen in the sampleSubmissionDF.head() output below. The id is a concatenation of the air_store_id and the calendar date. 

Our current trainDF does not have such concatenation, so we have to undo the concatenation, and later redo it after making a prediction for visitors.

The sample submission file has '0' entered for visitors. This column has to be dropped. Then, a new column will be added with our predictions of number of visitors.

In [7]:
col = [c for c in grouped2 if c not in ['air_store_id', 'visit_year','visit_month','visit_day', 'visitors','holiday_flg','day_of_week']]
print (col)

#model1 = neighbors.KNeighborsRegressor(n_jobs=-1,n_neighbors=2)

model1=ensemble.RandomForestRegressor(max_depth=5,random_state=1)

model1.fit(grouped2[col],np.log1p(grouped2['visitors'].values))


['day_of_week_codes', 'air_genre_name', 'latitude', 'longitude', 'reserve_visitors', 'median_visitors']


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=1, verbose=0, warm_start=False)

In [8]:
grouped2['mypredictions']= model1.predict(grouped2[col])
grouped2['mypredictions']=np.expm1(grouped2['mypredictions']).clip(lower=0.)

In [9]:
sampleSubmissionDF['air_store_id'] = sampleSubmissionDF.id.map(lambda x:'_'.join(x.split('_')[:-1]))

In [10]:
sampleSubmissionDF['calendar_date'] = sampleSubmissionDF.id.map(lambda x:x.split('_')[2])

In [11]:
sampleSubmissionDF.head()

,id,visitors,air_store_id,calendar_date
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27


In [12]:
sampleSubmissionDF['calendar_date'] = pd.DatetimeIndex(sampleSubmissionDF['calendar_date'])

In [13]:
dateInfoDF['day_of_week'] = dateInfoDF['day_of_week'].astype('category')
dateInfoDF.dtypes
dateInfoDF['day_of_week_codes'] = dateInfoDF['day_of_week'].cat.codes
dateInfoDF.head()

,calendar_date,day_of_week,holiday_flg,calendar_year,calendar_month,calendar_day,day_of_week_codes
0,2016-01-01,Friday,1,2016,1,1,0
1,2016-01-02,Saturday,1,2016,1,2,2
2,2016-01-03,Sunday,1,2016,1,3,3
3,2016-01-04,Monday,0,2016,1,4,1
4,2016-01-05,Tuesday,0,2016,1,5,5


In [14]:
sampleSubmissionDF = sampleSubmissionDF.merge(dateInfoDF,on ='calendar_date', how='left')

In [15]:
sampleSubmissionDF.drop(['calendar_month', 'calendar_day','calendar_date','holiday_flg','calendar_year','day_of_week'], axis=1, inplace=True)
sampleSubmissionDF.head()

,id,visitors,air_store_id,day_of_week_codes
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,3
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,1
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,5
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,6
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,4


In [16]:
sampleSubmissionDF = sampleSubmissionDF.merge(grouped2,on=['air_store_id','day_of_week_codes'],how='left')

In [17]:
sampleSubmissionDF.drop(['day_of_week_codes'],axis=1,inplace=True)
sampleSubmissionDF.drop(['visitors_x'],axis=1,inplace=True)

In [18]:
sampleSubmissionDF.drop(['visitors_y'],axis=1,inplace=True)
sampleSubmissionDF.drop(['air_store_id'],axis=1,inplace=True)
sampleSubmissionDF.drop(['reserve_visitors'],axis=1,inplace=True)
sampleSubmissionDF.drop(['air_genre_name'],axis=1,inplace=True)
sampleSubmissionDF.drop(['latitude'],axis=1,inplace=True)
sampleSubmissionDF.drop(['longitude'],axis=1,inplace=True)
sampleSubmissionDF.drop(['median_visitors'],axis=1,inplace=True)

In [19]:
sampleSubmissionDF['mypredictions'].fillna(0,inplace=True)

In [20]:
sampleSubmissionDF.to_csv('prediction1.csv',float_format='%.4f',index=None)

In [23]:
sampleSubmissionDF.tail(50)


,id,mypredictions
31969,air_ffcc2d5087e1b476_2017-05-21,0.0
31970,air_ffcc2d5087e1b476_2017-05-22,0.0
31971,air_ffcc2d5087e1b476_2017-05-23,0.0
31972,air_ffcc2d5087e1b476_2017-05-24,0.0
31973,air_ffcc2d5087e1b476_2017-05-25,0.0
31974,air_ffcc2d5087e1b476_2017-05-26,0.0
31975,air_ffcc2d5087e1b476_2017-05-27,0.0
31976,air_ffcc2d5087e1b476_2017-05-28,0.0
31977,air_ffcc2d5087e1b476_2017-05-29,0.0
31978,air_ffcc2d5087e1b476_2017-05-30,0.0
